# Module Imports

In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy.interpolate import griddata


In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

In [3]:
## IMPORT REGRESSSION MODULES
from sys import platform,path
import getpass
username = getpass.getuser()
if platform == 'darwin':
    module_path = '/Users/'+username+'/catkin_ws/src/crazyflie_simulation/src/Projects/IROS_DataAnalysis'
else:
    module_path = '/home/'+username+'/catkin_ws/src/crazyflie_simulation/src/Projects/IROS_DataAnalysis'
path.append(module_path)

from DataVisualizer.LinReg import plot_3d_data,linreg,reg_3d,reg_2d,color_plot

# DataFrame

In [4]:
## FULL DATAFRAME
df_raw = pd.read_csv("NL_2-Policy_Summary.csv")

## DROP EXTRANEOUS DATAPOINTS
df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vz_d']>=0)].index)
df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vx_d']>=0.0)].index)


## GROUP AND AVERAGE RAW DATA BY VEL IC
df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## CLEAN RAW DATA
df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df['My_d'] = df['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max moment
# df = df.drop(df[(df['vz_d']<= 2.0) & (df['vx_d']<= 0.5)].index) # Drop corner with no successful landings
df = df.dropna()

## MAXIMUM DATAFRAME
idx = df.groupby(['vz_d','vx_d'])['landing_rate'].transform(max) == df['landing_rate']
df_max = df[idx].reset_index()

## MY OUTLIER REMOVED DATAFRAME
df_max = df_max.query(f"My_d<={7.7}")
df_max = df_max.query(f"RREV_threshold>{3.5}") # Remove outlier data

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

# Landing Rate Data

In [5]:
## LANDING RATE DATA (MAX)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

x,y,z = 'vx_d', 'vz_d', 'landing_rate'
# DEFINE VARIABLES
X = df_max[x]
Y = df_max[y]
Z = df_max[z]


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)
ax.set_ylim(1.5,4)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Max Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.5752
Equation:  landing_rate  =  0.675196  +  0.150662 * vx_d  +  -0.022465 * vz_d


In [7]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.5641
Equation:  landing_rate  =  0.612378  +  0.151419 * vx_d


vz_d results:
r2 =  0.0174
Equation:  landing_rate  =  0.906584  +  -0.028147 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [8]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
x,y,z = 'vx_d','vz_d','RREV_threshold'

X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max[z]


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
#ax.plot_trisurf(X,Y,ypred,alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")



# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Max Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.4696
Equation:  RREV_threshold  =  5.138773  +  0.089427 * vx_d  +  -0.27434 * vz_d


In [10]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.0594
Equation:  RREV_threshold  =  4.371628  +  0.098674 * vx_d


vz_d results:
r2 =  0.4208
Equation:  RREV_threshold  =  5.276116  +  -0.277712 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


# DEFINE VARIABLES
x,y,z = 'RREV_threshold','OF_y','impact_tdelta'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
# # fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Max Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.6826
Equation:  impact_tdelta  =  0.727549  +  -0.109948 * RREV_threshold  +  -0.001233 * OF_y


In [13]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

RREV_threshold results:
r2 =  0.6806
Equation:  impact_tdelta  =  0.71569  +  -0.106598 * RREV_threshold


OF_y results:
r2 =  0.1368
Equation:  impact_tdelta  =  0.257703  +  0.008728 * OF_y


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [14]:
## Define Dataframe

#df_temp = df.query(f"landing_rate>={0.1}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


x,y,z = 'vx_d','vz_d','My_d'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
pcm = ax.scatter(X,Y,Z)



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Max Data)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.0494
Equation:  My_d  =  6.037378  +  -0.148703 * vx_d  +  -0.4401 * vz_d


In [16]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.0046
Equation:  My_d  =  4.806711  +  -0.133869 * vx_d


vz_d results:
r2 =  0.0437
Equation:  My_d  =  5.808999  +  -0.434492 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"impact_eul<={-60}")  
#df_temp = df_max

# DEFINE VARIABLES
x,y,z = 'vx_d','vz_d','impact_eul'
X = df_temp[x]
Y = df_temp[y]
Z = -df_temp[z]
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle [deg]')

ax.set_title('Impact Angle vs IC (Max Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.1372
Equation:  impact_eul  =  90.14191  +  6.927189 * vx_d  +  7.230499 * vz_d


In [19]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.0671
Equation:  impact_eul  =  110.360796  +  6.683484 * vx_d


vz_d results:
r2 =  0.0652
Equation:  impact_eul  =  100.780746  +  6.969253 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RAW POLICY RELATION

In [24]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


# DEFINE VARIABLES
x,y,z = 'RREV_threshold','OF_y','My_d'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.226
Equation:  My_d  =  -6.773708  +  2.622295 * RREV_threshold  +  0.168483 * OF_y


In [26]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

RREV_threshold results:
r2 =  0.1988
Equation:  My_d  =  -5.153858  +  2.164627 * RREV_threshold


OF_y results:
r2 =  0.0061
Equation:  My_d  =  4.432222  +  -0.069107 * OF_y


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …